In [1]:
import ray
import random, logging
import xml.etree.ElementTree as etree

In [2]:
def parse_post(xml):
    return etree.fromstring(xml)

In [3]:
posts = [
  '<row Id="1" Title="Eliciting priors from experts" />',
  '<row Id="2" Title="What is normality?" />',
  '<row Id="3" Title="What are some valuable Statistical Analysis open source projects?" />',
  '<row Id="4" Title="Assessing the significance of differences in distributions" />',
  '<row Id="5" Title="The Two Cultures: statistics vs. machine learning?" />',
  '<row Id="6" Title="Locating freely available data samples" />',
  '<row Id="7" Title="Forecasting demographic census" />',
  '<row Id="8" Title="Multivariate Interpolation Approaches" />',
  '<row Id="9" Title="How can I adapt ANOVA for binary data?" />'
]

In [4]:
[ parse_post(xml) for xml in posts ]

[<Element 'row' at 0x7fdbda368950>,
 <Element 'row' at 0x7fdbda3680e0>,
 <Element 'row' at 0x7fdbda3689f0>,
 <Element 'row' at 0x7fdbda368a90>,
 <Element 'row' at 0x7fdbda368ae0>,
 <Element 'row' at 0x7fdbda368b30>,
 <Element 'row' at 0x7fdbda368bd0>,
 <Element 'row' at 0x7fdbda368c20>,
 <Element 'row' at 0x7fdbda368c70>]

In [5]:
def parse_post(xml):
    post = etree.fromstring(xml)
    print(post.get('Id'))
    return post

In [6]:
[ parse_post(xml) for xml in posts ]

1
2
3
4
5
6
7
8
9


[<Element 'row' at 0x7fdbda372770>,
 <Element 'row' at 0x7fdbda3725e0>,
 <Element 'row' at 0x7fdbda372950>,
 <Element 'row' at 0x7fdbda372a40>,
 <Element 'row' at 0x7fdbda372a90>,
 <Element 'row' at 0x7fdbda3729f0>,
 <Element 'row' at 0x7fdbda372900>,
 <Element 'row' at 0x7fdbda372ae0>,
 <Element 'row' at 0x7fdbda372310>]

In [7]:
# Start Ray. If you're connecting to an existing cluster, you would use
# ray.init(address=<cluster-address>) instead.
ray.init(address='ray://192.168.2.133:10001')

ClientContext(dashboard_url='127.0.0.1:8265', python_version='3.8.10', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', protocol_version='2022-03-16', _num_clients=2, _context_to_restore=<ray.util.client._ClientContext object at 0x7fdbdecca700>)

In [8]:
@ray.remote
def parse_post(xml):
    post = etree.fromstring(xml)
    print(post.get('Id'))
    return post

In [9]:
future = parse_post.remote(posts[0])

(parse_post pid=21391) 1


In [10]:
ray.get(future)

<Element 'row' at 0x7fdbda2f51d0>

In [11]:
futures = [parse_post.remote(xml) for xml in posts ]

(parse_post pid=21391) 1
(parse_post pid=21391) 2
(parse_post pid=21391) 3
(parse_post pid=21391) 4
(parse_post pid=21391) 5
(parse_post pid=21391) 6
(parse_post pid=21391) 7
(parse_post pid=21391) 8
(parse_post pid=21391) 9


In [12]:
futures

[ClientObjectRef(e5cbd90b7f1fb776ffffffffffffffffffffffff0200000001000000),
 ClientObjectRef(39088be3736e590affffffffffffffffffffffff0200000001000000),
 ClientObjectRef(ce868e48e2fa9a94ffffffffffffffffffffffff0200000001000000),
 ClientObjectRef(f81ec6ff838b16dbffffffffffffffffffffffff0200000001000000),
 ClientObjectRef(32b0eec39cfa87acffffffffffffffffffffffff0200000001000000),
 ClientObjectRef(80b655a2d9b04d40ffffffffffffffffffffffff0200000001000000),
 ClientObjectRef(0d12401e8fa9a714ffffffffffffffffffffffff0200000001000000),
 ClientObjectRef(0021899358e75e3cffffffffffffffffffffffff0200000001000000),
 ClientObjectRef(0b043f00f0a74796ffffffffffffffffffffffff0200000001000000)]

In [13]:
ray.get(futures)

[<Element 'row' at 0x7fdbd827b310>,
 <Element 'row' at 0x7fdbd827b220>,
 <Element 'row' at 0x7fdbd827b400>,
 <Element 'row' at 0x7fdbd827b450>,
 <Element 'row' at 0x7fdbd827b4a0>,
 <Element 'row' at 0x7fdbd827b4f0>,
 <Element 'row' at 0x7fdbd827b540>,
 <Element 'row' at 0x7fdbd827b5e0>,
 <Element 'row' at 0x7fdbd827b630>]

In [ ]:
[ el.get('Id') for el in ray.get(futures) ]

In [ ]:
# similar to rdd.cache()
ref = ray.put("Jonathan")

In [ ]:
ray.get(ref)

In [ ]:
ref

## Actors

Scheme made them [concrete](https://dspace.mit.edu/handle/1721.1/5794). Erlang made them [useful](https://erlang.org/doc/getting_started/conc_prog.html). Akka made them [cool](https://akka.io/). And now Ray makes them [easy](https://docs.ray.io/en/latest/ray-overview/index.html)!

In [ ]:
!pip install faker

In [ ]:
@ray.remote
class Child(object):
    def __init__(self):
        from faker import Faker
        self.name = Faker().name()
        self.age = 1
        
    def grow(self):
        self.age += 1
        return self.age
    
    def greet(self):
        return (
            f'My name is {self.name} '
            f'and I am {self.age} years old'
        )

In [ ]:
children = [Child.remote() for i in range(10)]

In [ ]:
children

In [ ]:
futures = [ c.greet.remote() for c in children ]

In [ ]:
futures

In [ ]:
for future in ray.get(futures):
    print(future)

In [ ]:
for c in children:
    for _ in range(random.randint(1,10)):
        c.grow.remote()

In [ ]:
for future in ray.get([ c.greet.remote() for c in children ]):
    print(future)

In [ ]:
c = children[0]

In [ ]:
ray.get([c.grow.remote() for _ in range(5)])

In [ ]:
# actors stay around as long as they are in scope
# since nothing really goes out of scope in a notebook
# we have to manually terminate them
[ ray.kill(person) for person in children ]

In [ ]:
ray.shutdown()

## Simulating a pandemic

> note this is a toy model simulation, results should not be used to inform health decisions or personal behavior

### The SIR epidemic model:

$S(t)$: susceptible individuals who have not yet been infected at time $t$

$I(t)$: number of infectious individuals at time $t$

$R(t)$: number of individuals who have recovered (and are immune) at time $t$

#### Parameters

$\beta$: probablity of transmitted the disease from an infected to a susceptible individual

$\gamma$: recovery rate ~ $\frac{1}{\text{duration of disease}}$

We will follow the [EMOD compartamental model](https://idmod.org/docs/emod/malaria/model-compartments.html) to simulate the SIR model as a series of discrete timesteps. For something like reinforcement learning, instead of disease dynamics you simulate actions in an environment/game.

In [ ]:
ray.init(logging_level=logging.ERROR)

In [ ]:
# parameters
b = 0.5
b_0 = 0.2
g = 0.2
dim = 5

In [ ]:
@ray.remote
class Person(object):
    def __init__(self, i):
        self.index = i
        self.state = 'i' if random.random() < b_0 else 's'
        self.x = random.randint(0, dim)
        self.y = random.randint(0, dim)
        
    def location(self):
        return (self.x, self.y)
    
    def health(self):
        return self.state
    
    def index(self):
        return self.index
    
    def status(self):
        return f"Individual {self.index} at {self.location()} is currently {self.state}"
       
    def walk(self):
        if self.state == 'i':
            if random.random() < g:
                print(f"{self.index} has recovered ⚕️")
                self.state = 'r'

        self.x += random.randint(-1, 1)
        self.y += random.randint(-1, 1)
        
        self.x = max(min(self.x, dim), 0)
        self.y = max(min(self.y, dim), 0)
        
    def contract(self):
        print(f"{self.index} has become sick 🤮")
        self.state = 'i'
        
    def interact(self, stranger):
        x, y = ray.get(stranger.location.remote())
        state = ray.get(stranger.health.remote())
        
        # is the stranger close to me
        if (abs(x - self.x) <= 1) and (abs(y - self.y) <= 1):
            # is either of us infected?
            if self.state == 'i' or state == 'i':
                # can either of us _get_ infected?
                if self.state == 's' or state == 's':
                    # which one of us can get the disease
                    contract = self.contract if self.state == 's' else stranger.contract.remote
                    
                    # roll the dice babeeeeee
                    if random.random() < b:
                        contract()

In [ ]:
people = [Person.remote(i) for i in range(15)]

In [ ]:
people

In [ ]:
ray.get([p.location.remote() for p in people])

In [ ]:
ray.get([p.health.remote() for p in people])

In [ ]:
from itertools import combinations

In [ ]:
for i in range(20):
    print(f'\nIteration {i}\n\n')
    for person in people:
        person.walk.remote()
        
    pairs = list(combinations(people, 2))
    
    for p1, p2 in pairs:
        p1.interact.remote(p2)

In [ ]:
for person in people:
    print(ray.get(person.status.remote()))